# DPO and Alternatives: Simpler Paths to Alignment

What if we could skip the reward model and train directly on preferences?

## What You'll Learn

By the end of this notebook, you'll understand:
- The shortcut analogy: why DPO is revolutionary
- The math behind DPO (it's elegant!)
- Implementing DPO loss from scratch
- When to use DPO vs RLHF
- Other alternatives: RLAIF, Constitutional AI, ORPO
- Using TRL's DPOTrainer

**Prerequisites:** Notebooks 1-3 (RLHF basics, Reward Modeling, PPO for LLMs)

**Time:** ~30 minutes

---
## The Big Picture: The Shortcut Analogy

```
    ┌────────────────────────────────────────────────────────────────┐
    │          THE SHORTCUT ANALOGY                                  │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  Imagine you want to get from A to C...                       │
    │                                                                │
    │  RLHF ROUTE (PPO):                                            │
    │    A → B → C                                                  │
    │                                                                │
    │    A: Preference data (human labels)                         │
    │    B: Train reward model (intermediate step)                  │
    │    C: Train policy with PPO (final destination)              │
    │                                                                │
    │    Problems:                                                  │
    │    - Need to train 2 models                                  │
    │    - PPO is unstable                                         │
    │    - Complex pipeline                                        │
    │                                                                │
    │  DPO SHORTCUT:                                                │
    │    A ─────────────→ C                                        │
    │                                                                │
    │    Skip the reward model entirely!                           │
    │    Go directly from preferences to aligned model!            │
    │                                                                │
    │  THE KEY INSIGHT:                                             │
    │    The optimal policy under RLHF has a closed-form!          │
    │    We can derive what RL would learn... analytically!        │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt
from matplotlib.patches import FancyBboxPatch, Circle, FancyArrowPatch
import warnings
warnings.filterwarnings('ignore')

# Visualize RLHF vs DPO
fig, axes = plt.subplots(1, 2, figsize=(14, 7))

# Left: RLHF Pipeline
ax1 = axes[0]
ax1.set_xlim(0, 10)
ax1.set_ylim(0, 10)
ax1.axis('off')
ax1.set_title('RLHF Pipeline\n(3 Stages)', fontsize=14, fontweight='bold', color='#d32f2f')

# Stage boxes
stages_rlhf = [
    (1.5, 7.5, 'Stage 1:\nPreferences', '#e3f2fd', '#1976d2'),
    (1.5, 4.5, 'Stage 2:\nReward Model', '#fff3e0', '#f57c00'),
    (1.5, 1.5, 'Stage 3:\nPPO Training', '#ffcdd2', '#d32f2f'),
]

for x, y, text, fcolor, ecolor in stages_rlhf:
    box = FancyBboxPatch((x, y), 7, 2, boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax1.add_patch(box)
    ax1.text(x + 3.5, y + 1, text, ha='center', va='center', fontsize=10)

# Arrows
ax1.annotate('', xy=(5, 6.5), xytext=(5, 7.4),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))
ax1.annotate('', xy=(5, 3.5), xytext=(5, 4.4),
             arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# Complexity indicators
ax1.text(9, 8.5, '😓', fontsize=16)
ax1.text(9, 5.5, '😰', fontsize=16)
ax1.text(9, 2.5, '😵', fontsize=16)

# Right: DPO Pipeline
ax2 = axes[1]
ax2.set_xlim(0, 10)
ax2.set_ylim(0, 10)
ax2.axis('off')
ax2.set_title('DPO Pipeline\n(1 Stage!)', fontsize=14, fontweight='bold', color='#388e3c')

# Just two boxes
box1 = FancyBboxPatch((1.5, 7), 7, 2, boxstyle="round,pad=0.1",
                       facecolor='#e3f2fd', edgecolor='#1976d2', linewidth=2)
ax2.add_patch(box1)
ax2.text(5, 8, 'Preference Data', ha='center', va='center', fontsize=10)

box2 = FancyBboxPatch((1.5, 2), 7, 2, boxstyle="round,pad=0.1",
                       facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax2.add_patch(box2)
ax2.text(5, 3, 'Direct Training!\n(One supervised loss)', ha='center', va='center', fontsize=10)

# Big arrow
ax2.annotate('', xy=(5, 4.1), xytext=(5, 6.9),
             arrowprops=dict(arrowstyle='->', lw=4, color='#388e3c'))

ax2.text(6.5, 5.5, 'SKIP\nSTAGES!', fontsize=12, fontweight='bold', color='#388e3c')

# Happy face
ax2.text(9, 5, '😎', fontsize=24)

plt.tight_layout()
plt.show()

print("\nDPO vs RLHF:")
print("  RLHF: 3 stages, 2 models, unstable PPO")
print("  DPO:  1 stage, 1 model, stable supervised learning!")

---
## The Math: From RLHF to DPO

```
    ┌────────────────────────────────────────────────────────────────┐
    │              THE MATHEMATICAL INSIGHT                          │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  STEP 1: RLHF Objective                                       │
    │    max E[RM(x,y)] - β × KL(π || π_ref)                        │
    │                                                                │
    │  STEP 2: The Optimal Policy (closed form!)                    │
    │    The solution to this optimization is:                      │
    │                                                                │
    │    π*(y|x) = π_ref(y|x) × exp(r(x,y) / β) / Z(x)             │
    │                                                                │
    │    where Z(x) is the normalization constant                   │
    │                                                                │
    │  STEP 3: Rearranging for Reward                               │
    │    If we know the optimal policy, we can recover the reward:  │
    │                                                                │
    │    r(x,y) = β × log(π*(y|x) / π_ref(y|x)) + β × log Z(x)     │
    │                                                                │
    │  STEP 4: The DPO Trick                                        │
    │    For preference pairs, the Z(x) terms CANCEL OUT!           │
    │                                                                │
    │    P(y_w > y_l) = σ(β × [log π(y_w|x)/π_ref(y_w|x)           │
    │                        - log π(y_l|x)/π_ref(y_l|x)])         │
    │                                                                │
    │  RESULT: We can train directly on preferences!                │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize the mathematical derivation

fig, ax = plt.subplots(figsize=(14, 10))
ax.set_xlim(0, 14)
ax.set_ylim(0, 12)
ax.axis('off')
ax.set_title('DPO Derivation: The Key Insight', fontsize=16, fontweight='bold')

# Step boxes
steps = [
    (0.5, 9.5, 'Step 1: RLHF Objective', 
     'max E[r(x,y)] - β × KL(π || π_ref)', '#e3f2fd', '#1976d2'),
    (0.5, 7, 'Step 2: Optimal Policy',
     'π*(y|x) ∝ π_ref(y|x) × exp(r(x,y)/β)', '#fff3e0', '#f57c00'),
    (0.5, 4.5, 'Step 3: Reward from Policy',
     'r(x,y) = β × log(π(y|x)/π_ref(y|x)) + C', '#e1bee7', '#7b1fa2'),
    (0.5, 2, 'Step 4: DPO Loss',
     'L = -log σ(β × [log π(y_w)/π_ref(y_w) - log π(y_l)/π_ref(y_l)])', '#c8e6c9', '#388e3c'),
]

for x, y, title, formula, fcolor, ecolor in steps:
    box = FancyBboxPatch((x, y), 10, 2, boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax.add_patch(box)
    ax.text(x + 5, y + 1.5, title, ha='center', fontsize=11, fontweight='bold', color=ecolor)
    ax.text(x + 5, y + 0.6, formula, ha='center', fontsize=10, family='monospace')

# Arrows
for y in [9.4, 6.9, 4.4]:
    ax.annotate('', xy=(5.5, y-0.3), xytext=(5.5, y),
                arrowprops=dict(arrowstyle='->', lw=2, color='#666'))

# Key insight
insight_box = FancyBboxPatch((11, 5), 2.8, 3, boxstyle="round,pad=0.1",
                              facecolor='#ffeb3b', edgecolor='#f57f17', linewidth=2)
ax.add_patch(insight_box)
ax.text(12.4, 7.3, 'KEY', ha='center', fontsize=10, fontweight='bold', color='#f57f17')
ax.text(12.4, 6.6, 'INSIGHT', ha='center', fontsize=10, fontweight='bold', color='#f57f17')
ax.text(12.4, 5.8, 'Z(x) terms', ha='center', fontsize=9)
ax.text(12.4, 5.3, 'cancel out!', ha='center', fontsize=9)

plt.tight_layout()
plt.show()

print("\nTHE GENIUS OF DPO:")
print("  The normalization constant Z(x) is intractable...")
print("  BUT when comparing pairs, it cancels out!")
print("  No reward model needed - just supervised learning!")

---
## The DPO Loss Function

```
    ┌────────────────────────────────────────────────────────────────┐
    │              DPO LOSS FUNCTION                                  │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  LOSS:                                                        │
    │                                                                │
    │    L_DPO = -E[ log σ( β × (                                   │
    │                   log π_θ(y_w|x)/π_ref(y_w|x)                 │
    │                 - log π_θ(y_l|x)/π_ref(y_l|x)                 │
    │               ) ) ]                                           │
    │                                                                │
    │  WHERE:                                                       │
    │    y_w = winning (preferred) response                        │
    │    y_l = losing (rejected) response                          │
    │    π_θ = policy being trained                                │
    │    π_ref = reference policy (frozen)                         │
    │    β = temperature parameter                                 │
    │                                                                │
    │  INTUITION:                                                   │
    │    "Increase probability of y_w relative to reference"       │
    │    "Decrease probability of y_l relative to reference"       │
    │    Both constrained to not drift too far from reference!     │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
def dpo_loss(policy_chosen_logps, policy_rejected_logps,
             reference_chosen_logps, reference_rejected_logps,
             beta=0.1):
    """
    Compute DPO loss.
    
    This is THE key function - it's remarkably simple!
    
    Args:
        policy_chosen_logps: Log probs of chosen response under policy
        policy_rejected_logps: Log probs of rejected response under policy
        reference_chosen_logps: Log probs of chosen under reference
        reference_rejected_logps: Log probs of rejected under reference
        beta: Temperature parameter (controls strength)
    
    Returns:
        loss: DPO loss value
        chosen_rewards: Implicit rewards for chosen
        rejected_rewards: Implicit rewards for rejected
    """
    # Compute "implicit rewards" 
    # These are what RLHF would learn, derived analytically!
    chosen_rewards = beta * (policy_chosen_logps - reference_chosen_logps)
    rejected_rewards = beta * (policy_rejected_logps - reference_rejected_logps)
    
    # DPO loss: maximize margin between chosen and rejected rewards
    loss = -F.logsigmoid(chosen_rewards - rejected_rewards).mean()
    
    return loss, chosen_rewards, rejected_rewards


# Demonstrate DPO loss
print("DPO LOSS DEMONSTRATION")
print("="*60)

# Simulated log probabilities (batch of 4 examples)
torch.manual_seed(42)
batch_size = 4

# Reference model (frozen) - same for both
reference_chosen_logps = torch.randn(batch_size) - 5  # Around -5
reference_rejected_logps = torch.randn(batch_size) - 5

print("\nScenario 1: Policy increases chosen, decreases rejected (GOOD!)")
policy_chosen_logps = reference_chosen_logps + 0.5  # Increased
policy_rejected_logps = reference_rejected_logps - 0.3  # Decreased
loss1, cr1, rr1 = dpo_loss(policy_chosen_logps, policy_rejected_logps,
                           reference_chosen_logps, reference_rejected_logps)
print(f"  Loss: {loss1.item():.4f} (lower is better)")
print(f"  Chosen rewards: {cr1.mean().item():.4f}")
print(f"  Rejected rewards: {rr1.mean().item():.4f}")

print("\nScenario 2: Policy increases both equally (BAD!)")
policy_chosen_logps = reference_chosen_logps + 0.5
policy_rejected_logps = reference_rejected_logps + 0.5
loss2, cr2, rr2 = dpo_loss(policy_chosen_logps, policy_rejected_logps,
                           reference_chosen_logps, reference_rejected_logps)
print(f"  Loss: {loss2.item():.4f}")
print(f"  Chosen rewards: {cr2.mean().item():.4f}")
print(f"  Rejected rewards: {rr2.mean().item():.4f}")

print("\nScenario 3: Policy prefers rejected over chosen (VERY BAD!)")
policy_chosen_logps = reference_chosen_logps - 0.5  # Decreased
policy_rejected_logps = reference_rejected_logps + 0.5  # Increased
loss3, cr3, rr3 = dpo_loss(policy_chosen_logps, policy_rejected_logps,
                           reference_chosen_logps, reference_rejected_logps)
print(f"  Loss: {loss3.item():.4f} (very high!)")
print(f"  Chosen rewards: {cr3.mean().item():.4f}")
print(f"  Rejected rewards: {rr3.mean().item():.4f}")

print("\n" + "="*60)

In [ ]:
# Visualize DPO loss landscape

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Loss vs reward margin
ax1 = axes[0]
margin = np.linspace(-3, 3, 100)  # chosen_reward - rejected_reward
loss = -np.log(1 / (1 + np.exp(-margin)) + 1e-10)  # -log sigmoid

ax1.plot(margin, loss, 'b-', linewidth=3)
ax1.axvline(x=0, color='gray', linestyle='--', alpha=0.5)

ax1.fill_between(margin, 0, loss, where=(margin < 0), alpha=0.2, color='red', 
                 label='Rejected preferred (bad)')
ax1.fill_between(margin, 0, loss, where=(margin > 0), alpha=0.2, color='green',
                 label='Chosen preferred (good)')

ax1.set_xlabel('Reward Margin (chosen - rejected)', fontsize=11)
ax1.set_ylabel('DPO Loss', fontsize=11)
ax1.set_title('DPO Loss vs Reward Margin', fontsize=12, fontweight='bold')
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.set_ylim(0, 4)

# Right: Effect of beta
ax2 = axes[1]
log_ratio = np.linspace(-2, 2, 100)  # log(π/π_ref) difference

for beta in [0.05, 0.1, 0.2, 0.5]:
    margin = beta * log_ratio
    loss = -np.log(1 / (1 + np.exp(-margin)) + 1e-10)
    ax2.plot(log_ratio, loss, linewidth=2, label=f'β = {beta}')

ax2.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
ax2.set_xlabel('Log Ratio Difference', fontsize=11)
ax2.set_ylabel('DPO Loss', fontsize=11)
ax2.set_title('Effect of β (Temperature)', fontsize=12, fontweight='bold')
ax2.legend()
ax2.grid(True, alpha=0.3)
ax2.set_ylim(0, 3)

plt.tight_layout()
plt.show()

print("\nBETA (β) CONTROLS:")
print("  • Higher β: More sensitive to preference differences")
print("  • Lower β: More forgiving, smoother gradients")
print("  • Typical values: 0.1 - 0.5")

---
## Training with DPO: Step by Step

```
    ┌────────────────────────────────────────────────────────────────┐
    │              DPO TRAINING WORKFLOW                             │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  INPUT DATA:                                                  │
    │    Preference pairs: (prompt, chosen, rejected)              │
    │                                                                │
    │  MODELS:                                                      │
    │    π_θ (policy): Being trained                               │
    │    π_ref (reference): Frozen copy of initial π_θ             │
    │                                                                │
    │  TRAINING LOOP:                                               │
    │    for batch in dataloader:                                  │
    │        # 1. Get log probs from both models                   │
    │        π_chosen = policy(chosen_response)                    │
    │        π_rejected = policy(rejected_response)                │
    │        ref_chosen = reference(chosen_response)               │
    │        ref_rejected = reference(rejected_response)           │
    │                                                                │
    │        # 2. Compute DPO loss                                 │
    │        loss = dpo_loss(π_chosen, π_rejected,                 │
    │                        ref_chosen, ref_rejected)             │
    │                                                                │
    │        # 3. Standard gradient descent                        │
    │        loss.backward()                                       │
    │        optimizer.step()                                      │
    │                                                                │
    │  That's it! Just supervised learning with a special loss!    │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
class SimpleDPOTrainer:
    """
    A simplified DPO trainer for demonstration.
    
    In practice, use TRL's DPOTrainer.
    """
    
    def __init__(self, policy_model, reference_model, beta=0.1, lr=1e-5):
        self.policy = policy_model
        self.reference = reference_model
        self.beta = beta
        
        # Freeze reference model
        for param in self.reference.parameters():
            param.requires_grad = False
        
        self.optimizer = optim.AdamW(self.policy.parameters(), lr=lr)
    
    def compute_log_probs(self, model, input_ids):
        """
        Compute log probabilities for a sequence.
        
        In practice, this would use the actual tokens.
        Here we simulate with embeddings.
        """
        with torch.set_grad_enabled(model.training):
            logits = model(input_ids)
            # Simplified: just sum of logits as "log prob"
            log_probs = F.log_softmax(logits, dim=-1)
            return log_probs.sum(dim=-1).mean(dim=-1)
    
    def train_step(self, chosen_ids, rejected_ids):
        """
        Perform one DPO training step.
        
        Args:
            chosen_ids: Token ids for chosen responses
            rejected_ids: Token ids for rejected responses
        
        Returns:
            loss: DPO loss value
            metrics: Dictionary of metrics
        """
        self.optimizer.zero_grad()
        
        # Get log probs from policy
        self.policy.train()
        policy_chosen_logps = self.compute_log_probs(self.policy, chosen_ids)
        policy_rejected_logps = self.compute_log_probs(self.policy, rejected_ids)
        
        # Get log probs from reference (no grad)
        self.reference.eval()
        with torch.no_grad():
            ref_chosen_logps = self.compute_log_probs(self.reference, chosen_ids)
            ref_rejected_logps = self.compute_log_probs(self.reference, rejected_ids)
        
        # Compute DPO loss
        loss, chosen_rewards, rejected_rewards = dpo_loss(
            policy_chosen_logps, policy_rejected_logps,
            ref_chosen_logps, ref_rejected_logps,
            beta=self.beta
        )
        
        # Backward pass
        loss.backward()
        self.optimizer.step()
        
        # Compute metrics
        with torch.no_grad():
            reward_margin = (chosen_rewards - rejected_rewards).mean().item()
            accuracy = (chosen_rewards > rejected_rewards).float().mean().item()
        
        return loss.item(), {
            'reward_margin': reward_margin,
            'accuracy': accuracy,
            'chosen_reward': chosen_rewards.mean().item(),
            'rejected_reward': rejected_rewards.mean().item(),
        }


# Create simple model for demonstration
class SimpleModel(nn.Module):
    def __init__(self, vocab_size=1000, hidden_dim=128):
        super().__init__()
        self.embedding = nn.Embedding(vocab_size, hidden_dim)
        self.fc = nn.Linear(hidden_dim, vocab_size)
    
    def forward(self, x):
        return self.fc(self.embedding(x))


# Demonstrate training
print("DPO TRAINING DEMONSTRATION")
print("="*60)

# Create models
import copy
policy = SimpleModel()
reference = copy.deepcopy(policy)  # Frozen copy

# Create trainer
trainer = SimpleDPOTrainer(policy, reference, beta=0.1)

# Generate synthetic data
batch_size = 8
seq_len = 20

losses = []
accuracies = []

print("\nTraining for 100 steps...")
for step in range(100):
    # Random preference pairs (in reality, from dataset)
    chosen_ids = torch.randint(0, 1000, (batch_size, seq_len))
    rejected_ids = torch.randint(0, 1000, (batch_size, seq_len))
    
    loss, metrics = trainer.train_step(chosen_ids, rejected_ids)
    losses.append(loss)
    accuracies.append(metrics['accuracy'])
    
    if (step + 1) % 20 == 0:
        print(f"  Step {step+1}: Loss={loss:.4f}, Accuracy={metrics['accuracy']:.1%}")

print("\n" + "="*60)

In [ ]:
# Visualize training

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Left: Loss curve
ax1 = axes[0]
ax1.plot(losses, color='#f57c00', linewidth=2)
ax1.set_xlabel('Training Step', fontsize=11)
ax1.set_ylabel('DPO Loss', fontsize=11)
ax1.set_title('DPO Training Loss', fontsize=12, fontweight='bold')
ax1.grid(True, alpha=0.3)

# Right: Accuracy curve
ax2 = axes[1]
ax2.plot(accuracies, color='#4caf50', linewidth=2)
ax2.axhline(y=0.5, color='gray', linestyle='--', label='Random (50%)')
ax2.set_xlabel('Training Step', fontsize=11)
ax2.set_ylabel('Preference Accuracy', fontsize=11)
ax2.set_title('DPO Preference Prediction Accuracy', fontsize=12, fontweight='bold')
ax2.set_ylim(0.3, 1.0)
ax2.legend()
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\nDPO TRAINING CHARACTERISTICS:")
print("  • Stable loss decrease (no PPO instability!)")
print("  • Simple supervised learning dynamics")
print("  • Accuracy improves as model learns preferences")

---
## DPO vs RLHF: When to Use Which?

```
    ┌────────────────────────────────────────────────────────────────┐
    │              DPO vs RLHF COMPARISON                            │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  USE DPO WHEN:                                                │
    │    ✓ You have good preference data                           │
    │    ✓ Simplicity is important                                 │
    │    ✓ You want stable training                                │
    │    ✓ Compute is limited                                      │
    │    ✓ One-shot alignment (no iteration)                       │
    │                                                                │
    │  USE RLHF (PPO) WHEN:                                         │
    │    ✓ You need online learning (iterative improvement)        │
    │    ✓ Reward model needs to score new generations             │
    │    ✓ More control over reward function                       │
    │    ✓ Complex reward shaping is needed                        │
    │    ✓ Multi-objective optimization                            │
    │                                                                │
    │  CURRENT TREND (2024):                                        │
    │    Many practitioners now START with DPO!                    │
    │    Simpler, often as effective as RLHF.                      │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Create comparison visualization

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('DPO vs RLHF: Comparison', fontsize=16, fontweight='bold')

# RLHF column
rlhf_box = FancyBboxPatch((0.5, 1), 6, 7.5, boxstyle="round,pad=0.1",
                           facecolor='#fff3e0', edgecolor='#f57c00', linewidth=3)
ax.add_patch(rlhf_box)
ax.text(3.5, 8, 'RLHF (PPO)', ha='center', fontsize=14, fontweight='bold', color='#f57c00')

rlhf_items = [
    ('Stages', '3 (SFT→RM→PPO)'),
    ('Complexity', 'High'),
    ('Stability', 'Can be unstable'),
    ('Online Learning', '✓ Yes'),
    ('Memory', 'Higher (4 models)'),
    ('Flexibility', 'Very flexible'),
]

for i, (label, value) in enumerate(rlhf_items):
    y = 7 - i * 0.9
    ax.text(1, y, f'{label}:', fontsize=10, fontweight='bold')
    ax.text(3.2, y, value, fontsize=10)

# DPO column
dpo_box = FancyBboxPatch((7.5, 1), 6, 7.5, boxstyle="round,pad=0.1",
                          facecolor='#c8e6c9', edgecolor='#388e3c', linewidth=3)
ax.add_patch(dpo_box)
ax.text(10.5, 8, 'DPO', ha='center', fontsize=14, fontweight='bold', color='#388e3c')

dpo_items = [
    ('Stages', '1 (Direct!)'),
    ('Complexity', 'Low'),
    ('Stability', 'Very stable'),
    ('Online Learning', '✗ Offline only'),
    ('Memory', 'Lower (2 models)'),
    ('Flexibility', 'Less flexible'),
]

for i, (label, value) in enumerate(dpo_items):
    y = 7 - i * 0.9
    ax.text(8, y, f'{label}:', fontsize=10, fontweight='bold')
    ax.text(10.2, y, value, fontsize=10)

plt.tight_layout()
plt.show()

# Print comparison table
print("\nDETAILED COMPARISON TABLE")
print("="*60)
print(f"{'Aspect':<20} {'RLHF (PPO)':<20} {'DPO':<20}")
print("-"*60)

comparisons = [
    ('Training Stages', '3 stages', '1 stage'),
    ('Reward Model', 'Required', 'Not needed'),
    ('Stability', 'Can be unstable', 'Very stable'),
    ('Online Data', 'Supported', 'Not supported'),
    ('Memory (GPUs)', '4 models', '2 models'),
    ('Implementation', 'Complex', 'Simple'),
    ('Flexibility', 'High', 'Medium'),
    ('Performance', 'Excellent', 'Very good'),
]

for aspect, rlhf, dpo in comparisons:
    print(f"{aspect:<20} {rlhf:<20} {dpo:<20}")

print("="*60)

---
## Other Alternatives to RLHF

```
    ┌────────────────────────────────────────────────────────────────┐
    │              OTHER ALIGNMENT METHODS                           │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  RLAIF (RL from AI Feedback):                                 │
    │    Use AI to generate preferences instead of humans           │
    │    → Cheaper, faster, more scalable                          │
    │    → But may inherit AI biases                               │
    │                                                                │
    │  Constitutional AI (CAI):                                     │
    │    Model critiques and revises its own outputs               │
    │    Based on a set of principles/rules                        │
    │    → Self-improvement without human labels                   │
    │                                                                │
    │  ORPO (Odds Ratio Preference Optimization):                   │
    │    Combines SFT and preference learning in one step          │
    │    → Even simpler than DPO                                   │
    │    → No need for reference model during training             │
    │                                                                │
    │  IPO (Identity Preference Optimization):                      │
    │    Fixes some theoretical issues with DPO                    │
    │    → Better calibration                                      │
    │                                                                │
    │  KTO (Kahneman-Tversky Optimization):                        │
    │    Based on prospect theory from economics                   │
    │    → Can work with just thumbs up/down (no pairs!)          │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Visualize the landscape of alignment methods

fig, ax = plt.subplots(figsize=(14, 8))
ax.set_xlim(0, 14)
ax.set_ylim(0, 10)
ax.axis('off')
ax.set_title('Landscape of LLM Alignment Methods', fontsize=16, fontweight='bold')

# Methods as boxes
methods = [
    (1, 7, 'RLHF\n(PPO)', 'The OG', '#fff3e0', '#f57c00'),
    (4, 7, 'DPO', 'Simple & Effective', '#c8e6c9', '#388e3c'),
    (7, 7, 'ORPO', 'Combined SFT+DPO', '#bbdefb', '#1976d2'),
    (10, 7, 'KTO', 'No pairs needed', '#e1bee7', '#7b1fa2'),
    (2.5, 4, 'RLAIF', 'AI as annotator', '#ffcdd2', '#d32f2f'),
    (5.5, 4, 'Constitutional\nAI', 'Self-critique', '#b2dfdb', '#00796b'),
    (8.5, 4, 'IPO', 'Better calibration', '#fff9c4', '#fbc02d'),
]

for x, y, name, desc, fcolor, ecolor in methods:
    box = FancyBboxPatch((x, y), 2.5, 2, boxstyle="round,pad=0.1",
                          facecolor=fcolor, edgecolor=ecolor, linewidth=2)
    ax.add_patch(box)
    ax.text(x + 1.25, y + 1.4, name, ha='center', fontsize=9, fontweight='bold')
    ax.text(x + 1.25, y + 0.4, desc, ha='center', fontsize=8, color='#666')

# Timeline arrow
ax.annotate('', xy=(12, 1.5), xytext=(1, 1.5),
             arrowprops=dict(arrowstyle='->', lw=2, color='#333'))
ax.text(6.5, 0.8, 'Trend: Simpler → More efficient', ha='center', fontsize=11, style='italic')

plt.tight_layout()
plt.show()

print("\nKEY TRENDS IN LLM ALIGNMENT:")
print("  1. Moving away from complex RL (PPO)")
print("  2. Preference learning without reward models")
print("  3. Using AI to generate training signal")
print("  4. Simpler methods that work just as well")

---
## Using TRL's DPOTrainer

```
    ┌────────────────────────────────────────────────────────────────┐
    │              TRL DPOTrainer                                    │
    ├────────────────────────────────────────────────────────────────┤
    │                                                                │
    │  TRL provides a production-ready DPOTrainer!                  │
    │                                                                │
    │  KEY FEATURES:                                                │
    │    • Handles reference model automatically                   │
    │    • Supports LoRA/QLoRA for efficient training              │
    │    • Distributed training support                            │
    │    • Logging and evaluation built-in                         │
    │                                                                │
    │  DATA FORMAT:                                                 │
    │    {                                                          │
    │      "prompt": "Question here",                              │
    │      "chosen": "Good answer",                                │
    │      "rejected": "Bad answer"                                │
    │    }                                                          │
    │                                                                │
    └────────────────────────────────────────────────────────────────┘
```

In [ ]:
# Check TRL availability
try:
    from trl import DPOTrainer, DPOConfig
    TRL_AVAILABLE = True
    print("✓ TRL is installed!")
except ImportError:
    TRL_AVAILABLE = False
    print("✗ TRL not installed.")
    print("  Install with: pip install trl transformers")

# Show example code
print("\n" + "="*60)
print("TRL DPOTrainer EXAMPLE CODE")
print("="*60)

example_code = '''
from trl import DPOTrainer, DPOConfig
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig
from datasets import load_dataset

# 1. Load model and tokenizer
model_name = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# 2. Load preference dataset
# Format: {"prompt": ..., "chosen": ..., "rejected": ...}
dataset = load_dataset("Anthropic/hh-rlhf", split="train[:1000]")

# 3. Configure LoRA for efficient training
peft_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
)

# 4. Configure DPO training
training_args = DPOConfig(
    output_dir="dpo_model",
    beta=0.1,                        # KL penalty coefficient
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    learning_rate=5e-5,
    num_train_epochs=1,
    max_length=512,
    max_prompt_length=256,
    logging_steps=10,
)

# 5. Create trainer (reference model created automatically!)
trainer = DPOTrainer(
    model=model,
    args=training_args,
    train_dataset=dataset,
    tokenizer=tokenizer,
    peft_config=peft_config,  # Optional: use LoRA
)

# 6. Train!
trainer.train()

# 7. Save
trainer.save_model("dpo_model_final")
'''

print(example_code)
print("="*60)

---
## Summary: Key Takeaways

### DPO Key Formula

```
L_DPO = -log σ(β × [log π(y_w)/π_ref(y_w) - log π(y_l)/π_ref(y_l)])
```

### DPO vs RLHF

| Aspect | DPO | RLHF (PPO) |
|--------|-----|------------|
| Stages | 1 | 3 |
| Reward Model | Not needed | Required |
| Training | Supervised | Reinforcement |
| Stability | High | Can be unstable |
| Online Learning | No | Yes |

### Other Methods

| Method | Key Feature |
|--------|-------------|
| RLAIF | AI generates preferences |
| Constitutional AI | Self-critique |
| ORPO | Combined SFT + DPO |
| KTO | No pairs needed |

---
## Test Your Understanding

**1. What is the key mathematical insight behind DPO?**
<details>
<summary>Click to reveal answer</summary>
The optimal policy under the RLHF objective has a closed-form solution:
π*(y|x) ∝ π_ref(y|x) × exp(r(x,y)/β)

By rearranging this, we can express the reward in terms of log probabilities. When comparing pairs, the intractable normalization constant Z(x) cancels out, allowing direct training on preferences without learning a reward model!
</details>

**2. Why doesn't DPO need a reward model?**
<details>
<summary>Click to reveal answer</summary>
DPO learns the "implicit reward" directly from the log probability ratios:
r(x,y) = β × log(π(y|x)/π_ref(y|x))

Instead of training a separate model to predict rewards, DPO learns a policy where the probability ratios encode the reward signal. The preference data directly guides policy optimization.
</details>

**3. When should you choose RLHF over DPO?**
<details>
<summary>Click to reveal answer</summary>
Choose RLHF when:
- You need online learning (generate, collect feedback, iterate)
- You want to use the reward model for other purposes
- You need complex reward shaping or multi-objective optimization
- You want maximum flexibility in the reward function

DPO is offline-only - it can't improve based on new generations.
</details>

**4. What does β control in DPO?**
<details>
<summary>Click to reveal answer</summary>
β is the temperature/KL penalty coefficient:
- Higher β: Stronger signal from preferences, more aggressive changes
- Lower β: Weaker signal, stay closer to reference model

It's the same β as in RLHF! It controls how much the model can deviate from the reference policy.
</details>

**5. What is RLAIF and why is it useful?**
<details>
<summary>Click to reveal answer</summary>
RLAIF (RL from AI Feedback) uses an AI model to generate preferences instead of humans:
- Much cheaper and faster than human annotation
- Can generate millions of preference pairs
- Scales easily

Drawback: May inherit biases from the AI labeler. Works best when combined with some human oversight.
</details>

---
## What's Next?

Now you understand DPO and alternatives! In the next notebook, we'll explore the **TRL Library** in depth for practical RLHF/DPO training.

**Continue to:** [Notebook 5: TRL Library Tutorial](05_trl_library_tutorial.ipynb)

---

*DPO: "Skip the middleman, go straight to alignment!"*